# Scrape County Commission Notes

Scraping the Shelby County Commission's meeting notes for how the current Memphis 
mayoral candidates voted on ordinances that were adopted.

In [49]:
from pathlib import Path

import bs4
import openpyxl
import pandas as pd
import regex
import requests
from tqdm import tqdm

In [50]:
# Read in the ordinance summaries downloaded from
# https://library.municode.com/tn/shelby_county/ordinances/code_of_ordinances
ordinances = []
with open('cc-ordinances.txt') as f:
    for _ in range(124):
        ordinances.append(
            dict(
                adopted=f.readline().split()[-1],
                ordinance=int(f.readline().split()[-1]),
                title=f.readline().strip(),
            ))
ordinances = pd.DataFrame(ordinances).set_index('ordinance')
ordinances.adopted = pd.to_datetime(ordinances.adopted).astype(str)
ordinances.head()

,adopted,title
ordinance,,
409,2011-10-17,"An Ordinance Amending Chapter 24, Article X, R..."
408,2011-08-08,"An Ordinance Amending Chapter 38, Article II, ..."
407,2011-07-25,"An Ordinance Amending Section 1-4, to Make it ..."
406,2011-07-11,"An Ordinance Amending Chapter 16, to Prohibit ..."
405,2011-07-11,An Ordinance Fixing the Tax Rate for the Tax Y...


## Get links to meeting minutes

From table copied from [archive site](https://www.shelbycountytn.gov/1218/Archived-Meeting-Audio) into Excel

In [51]:
archive_xlsx = 'cc-meeting-archive.xlsx'
archive_df = pd.read_excel(archive_xlsx)
archive_df.Date = pd.to_datetime(archive_df.Date)

wb = openpyxl.load_workbook(archive_xlsx)
ws = wb['Sheet1']

meeting_links = {}
for idx, row in archive_df.iterrows():
    if row.Name.startswith('Commission Meeting') and row.Date < pd.to_datetime(
            '2022-05-23') and not pd.isna(row.Minutes):
        meeting_links[str(row.Date.date())] = ws.cell(
            row=idx + 2, column=5).hyperlink.target
len(meeting_links)

345

## Download notes for each meeting

But just the ones referenced by the ordinances

In [52]:
ordinances = ordinances.join(pd.Series(meeting_links, name='url'),
                             on='adopted')


In [53]:
# Almost all missing ones are before the cutoff date
display(ordinances[ordinances['url'].isna()])
ordinances = ordinances.dropna(subset=['url'])

,adopted,title,url
ordinance,,,
499,2018-12-18,ORDINANCE AMENDING THE SHELBY COUNTY CODE OF O...,NaN
532,2021-11-08,ORDINANCE REAPPORTIONING THE DISTRICTS OF THE ...,NaN
548,2022-12-19,JOINT ORDINANCE OF THE BOARD OF COUNTY COMMISS...,NaN
547,2022-11-14,ORDINANCE TO AMEND THE REGULATIONS GOVERNING T...,NaN
546,2022-10-31,ORDINANCE TO AMEND THE SHELBY COUNTY CODE OF O...,NaN
545,2022-10-17,ORDINANCE BY THE BOARD OF COUNTY COMMISSIONERS...,NaN
544,2022-08-08,AN ORDINANCE AMENDING THE SHELBY COUNTY CODE O...,NaN
543,2022-06-27,AN ORDINANCE TO PROVIDE FOR THE ESTABLISHMENT ...,NaN
542,2022-06-27,AN ORDINANCE CREATING A SPECIAL REVENUE FUND T...,NaN


In [55]:
# Download the notes - this takes a while
notes_dir = Path('meeting-minutes')
notes_dir.mkdir(exist_ok=True)

for date, url in tqdm(
        ordinances.drop_duplicates(subset='url')[['adopted', 'url']].values):
    notes = notes_dir / f'{date}.txt'
    if notes.isfile():
        continue
    else:
        response = requests.get(url)
        response.raise_for_status()
        notes.write_text(response.text)

## Find relevant sections and extract votes

In [57]:
ordinance_votes = {}

for idx, row in ordinances.iterrows():
    if pd.isna(row.url):
        continue

    minutes = Path(f'cc-meeting-minutes/{row.adopted}.txt').read_text()
    soup = bs4.BeautifulSoup(minutes, 'html.parser')

    found = False
    text = ''
    for result in soup.find_all('div'):
        if 'ORDINANCE' in result.text:
            found = True
        if found:
            text += result.text
        if 'ABSTAINED' in text:
            break

    votes = {}
    for vote, voters in regex.findall(
            "([A-Z]+):\s+((?>Commissioner\s\w+,?\s?)+)", text):
        for voter in regex.findall('Commissioner\s(\w+)', voters):
            votes[voter] = vote.lower().rstrip('s')
    ordinance_votes[idx] = votes

In [58]:
# The time frame we care about doesn't go back far enough for JW Gibson's tenure as commissioner
pd.DataFrame(ordinance_votes).T.columns.sort_values()

Index(['Bailey', 'Basar', 'Billingsley', 'Bradford', 'Brooks', 'Bunker',
       'Burgess', 'Carpenter', 'Chism', 'Ford', 'Harvey', 'Jones', 'Lowery',
       'Mills', 'Milton', 'Morrison', 'Mulroy', 'Reaves', 'Ritz', 'Roland',
       'Sawyer', 'Shafer', 'Taylor', 'Thomas', 'Turner', 'Whaley', 'Wright'],
      dtype='object')

In [76]:
pd.DataFrame(ordinance_votes).T

,Harvey,Roland,Ritz,Thomas,Mulroy,Shafer,Burgess,Bunker,Chism,Brooks,...,Milton,Turner,Jones,Morrison,Whaley,Wright,Bradford,Mills,Lowery,Sawyer
409,aye,aye,aye,aye,aye,aye,aye,aye,aye,nay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,aye,aye,absent,absent,aye,aye,aye,aye,aye,absent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,abstained,aye,aye,aye,aye,aye,aye,absent,aye,absent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,aye,aye,aye,aye,absent,aye,nay,aye,aye,nay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,aye,aye,aye,aye,absent,aye,nay,aye,aye,nay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aye,...,aye,aye,aye,aye,aye,aye,aye,aye,aye,absent
517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aye,...,aye,aye,aye,aye,aye,aye,aye,aye,aye,absent
515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aye,...,aye,aye,aye,nay,aye,nay,aye,nay,aye,aye
538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nay,...,nay,nay,nay,absent,nay,aye,aye,aye,absent,absent


In [77]:
candidate_votes = (  # Grab only the candidates running with records
    pd.DataFrame(ordinance_votes).T[['Turner', 'Harvey']].dropna(
        axis='index',
        how='all').melt(value_name='vote',
                        var_name='candidate',
                        ignore_index=False).dropna().rename_axis('ordinance'))
candidate_votes


,candidate,vote
ordinance,,
453,Turner,aye
452,Turner,aye
462,Turner,aye
461,Turner,absent
460,Turner,aye
...,...,...
430,Harvey,absent
451,Harvey,aye
450,Harvey,aye


In [69]:
# Load manually annotated categories
tag_names = {
    'crime': 'Crime & Safety',
    'education': 'Public Education',
    'housing': 'Housing'
}
tags = pd.read_csv(
    'cc-ordinance-tags.tsv', sep='\t', header=1,
    index_col=1)['Unnamed: 3'].rename('tags').replace(tag_names).fillna('')

tags.value_counts()


                    77
Housing             24
Crime & Safety       9
Public Education     2
Name: tags, dtype: int64

In [83]:
# Export joined table
ordinances.join(tags).join(candidate_votes).assign(
    kind='County Commission ordinance').rename_axis('identifier').to_csv(
        'cc-votes.tsv', sep='\t')
